In [1]:
import pandas as pd

In [2]:
import gensim

In [3]:
df_read = pd.read_excel('source\homestay master.xlsx', header=0)
df_read = df_read[~df_read["ลักษณะที่พัก"].isnull()]

In [4]:
fix_features = df_read[["รหัส",'อาหารเช้า', 'ทำอาหารเองได้ภายใน', 'ทำอาหารเองได้ภายนอก', 'กางเต๊นท์',
       'พืนที่สวน', 'สัตว์เลี้ยงในบริเวณ', 'แหล่งน้ำ', 'สระว่ายน้ำ',
       'ที่จอดรถ','WiFi ในห้อง']]
fix_features.set_index(["รหัส"],inplace=True)

In [22]:
fix_features.head(3)

,อาหารเช้า,ทำอาหารเองได้ภายใน,ทำอาหารเองได้ภายนอก,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,WiFi ในห้อง
รหัส,,,,,,,,,,
PKG0001,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
PKG0002,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
PKG0004,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [6]:
df_filter = df_read.copy()
df_filter.set_index(["รหัส"],inplace=True)
df_filter = df_filter.fillna('n/a')

In [7]:
df_filter['Platform ขาย'] = df_filter['Platform ขาย'].apply(lambda x: x.replace(' ','').split(","))
df_filter['Platform Review'] = df_filter['Platform Review'].apply(lambda x: x.replace(' ','').split(","))
df_filter['ลักษณะที่พัก'] = df_filter['ลักษณะที่พัก'].apply(lambda x: x.replace(' ','').split(","))
df_filter['สไตล์'] = df_filter['สไตล์'].apply(lambda x: x.replace(' ','').split(","))
df_filter['จุดขายพิเศษ'] = df_filter['จุดขายพิเศษ'].apply(lambda x: x.replace(' ','').split(","))
df_filter['อรรถประโยชน์'] = df_filter['อรรถประโยชน์'].apply(lambda x: x.replace(' ','').split(","))

In [8]:
df_filter.head(3)

,ชื่ออสังหาริมทรัพย์ตัวอย่าง,Lat-Lon,ตำบล,โซน,ประเภท,Rating (Google Map),จำนวนคน Rate (GM),Platform ขาย,Platform Review,ราคาเริ่มต้น/คืน,...,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,อรรถประโยชน์,WiFi ในห้อง,จุดขายพิเศษ,Unnamed: 28
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,Angel Valley,"14.469428283417516, 101.57710441380485",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.9,18.0,[Airbnb],[ใกล้ไกล],15000.0,...,0.0,1.0,1.0,1.0,0.0,1.0,"[ATV, จักรยาน, BBQ, Clubhouse, คาราโอเกะ, พักร...",1.0,[n/a],n/a
PKG0002,Antani Home Khaoyai,"14.47240737839758, 101.57533605260043",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.6,26.0,"[Agoda, Booking, Tripadviser, Facebook]","[ชิลไปไหน, Familypool]",2000.0,...,0.0,1.0,0.0,1.0,1.0,1.0,"[พูลวิลล่า, Clubhouse, คาราโอเกะ, BBQ, หมูกระทะ]",1.0,[n/a],n/a
PKG0004,Barn House Pong Ta Long,"14.471865480259176, 101.58365880642997",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,5.0,1.0,"[Agoda, Booking, Bedandbreakfast]",[n/a],10000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,"[ห้ามจัดปาร์ตี้, BBQ, หมูกระทะ]",0.0,[n/a],n/a


In [9]:
def countElem(df_in, fieldName):
    dictionary = gensim.corpora.Dictionary(df_in[fieldName])
    df_in[fieldName] = df_in[fieldName].map(dictionary.doc2bow)
    df_in[fieldName] = df_in[fieldName].apply(lambda x:[(dictionary[id_], frequence) for id_, frequence in x])
    return df_in

In [10]:
df_filter = countElem(df_filter, 'Platform ขาย')
df_filter = countElem(df_filter, 'Platform Review')
df_filter = countElem(df_filter, 'ลักษณะที่พัก')
df_filter = countElem(df_filter, 'สไตล์')
df_filter = countElem(df_filter, 'จุดขายพิเศษ')
df_filter = countElem(df_filter, 'อรรถประโยชน์')

In [11]:
df_filter.head(3)

,ชื่ออสังหาริมทรัพย์ตัวอย่าง,Lat-Lon,ตำบล,โซน,ประเภท,Rating (Google Map),จำนวนคน Rate (GM),Platform ขาย,Platform Review,ราคาเริ่มต้น/คืน,...,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,สระว่ายน้ำ,ที่จอดรถ,อรรถประโยชน์,WiFi ในห้อง,จุดขายพิเศษ,Unnamed: 28
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,Angel Valley,"14.469428283417516, 101.57710441380485",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.9,18.0,"[(Airbnb, 1)]","[(ใกล้ไกล, 1)]",15000.0,...,0.0,1.0,1.0,1.0,0.0,1.0,"[(ATV, 1), (BBQ, 1), (Clubhouse, 1), (คาราโอเก...",1.0,"[(n/a, 1)]",n/a
PKG0002,Antani Home Khaoyai,"14.47240737839758, 101.57533605260043",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,4.6,26.0,"[(Agoda, 1), (Booking, 1), (Facebook, 1), (Tri...","[(Familypool, 1), (ชิลไปไหน, 1)]",2000.0,...,0.0,1.0,0.0,1.0,1.0,1.0,"[(BBQ, 1), (Clubhouse, 1), (คาราโอเกะ, 1), (พู...",1.0,"[(n/a, 1)]",n/a
PKG0004,Barn House Pong Ta Long,"14.471865480259176, 101.58365880642997",โป่งตาลอง,เขาใหญ่ตะวันออก,Resort/Hotel,5.0,1.0,"[(Agoda, 1), (Booking, 1), (Bedandbreakfast, 1)]","[(n/a, 1)]",10000.0,...,0.0,1.0,0.0,0.0,0.0,1.0,"[(BBQ, 1), (หมูกระทะ, 1), (ห้ามจัดปาร์ตี้, 1)]",0.0,"[(n/a, 1)]",n/a


In [35]:
def unpivotCount(df_in, fieldName):
    df_out = pd.concat([pd.DataFrame(s,columns=['w','c']).set_index('w') for s in df_in[fieldName]], \
          axis=1, sort=False).fillna(0).T.set_index(df_in.index)
    #df_out.reset_index(inplace=True)
    return df_out

Generate ข้อมูลตาม Domain ที่สนใจ เตรียมเอาไปทำ Conceptual Map แยก

In [36]:
df_platform = unpivotCount(df_filter, 'Platform ขาย')
df_platform_review = unpivotCount(df_filter, 'Platform Review')
df_acc = unpivotCount(df_filter, 'ลักษณะที่พัก')
df_style = unpivotCount(df_filter, 'สไตล์')
df_highlight = unpivotCount(df_filter, 'จุดขายพิเศษ')
df_util = unpivotCount(df_filter, 'อรรถประโยชน์')

รวมข้อมูล กรณีทำ MDS รวบทุก Domain

In [40]:
df_all = pd.concat([df_platform, df_platform_review,df_acc,df_style,df_highlight,df_util,fix_features]). \
            groupby(['รหัส']).sum()

In [41]:
df_all = df_all.drop('n/a', axis=1)

In [43]:
df_all.head(3)

,Airbnb,Agoda,Booking,Facebook,Tripadviser,Bedandbreakfast,www,PriceLine,Instagram,Findhotel,...,สระว่ายน้ำปรับอุณหภูมิได้,อาหารเช้า,ทำอาหารเองได้ภายใน,ทำอาหารเองได้ภายนอก,กางเต๊นท์,พืนที่สวน,สัตว์เลี้ยงในบริเวณ,แหล่งน้ำ,ที่จอดรถ,WiFi ในห้อง
รหัส,,,,,,,,,,,,,,,,,,,,,
PKG0001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
PKG0002,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0
PKG0004,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


Mapping Similarity

In [45]:
item_item_matrix = pd.DataFrame(index=df_all.columns,columns=df_all.columns).fillna(0)

In [46]:
def jaccard_index(df,i,j):
    a = pd.DataFrame(df.iloc[:,i])
    b = pd.DataFrame(df.iloc[:,j])
    slice_df = pd.concat([a,b], axis=1, join="inner")
    slice_df.columns = ['A','B']
    a_in_b = slice_df[(slice_df.A>0)&(slice_df.B>0)]
    a_occur = slice_df[(slice_df.A>0)]
    b_occur = slice_df[(slice_df.B>0)]
    jacc_index = a_in_b.shape[0]/(a_occur.shape[0]+b_occur.shape[0]-a_in_b.shape[0])
    return jacc_index

In [ ]:
# Co-occurrence Similarity
for i in range(0,len(item_item_matrix.columns)):
    for j in range(0,len(item_item_matrix.columns)):
        item_item_matrix.iloc[i,j] = jaccard_index(df_all,i,j)

In [ ]:
item_item_matrix.head()

In [ ]:
item_item_matrix.to_excel('coocMatrix.xlsx', index=True)